In [ ]:
import librosa
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import librosa.display
import IPython.display as ipd
from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
from pydub import AudioSegment
from scipy.io import wavfile
from audio2numpy import open_audio

#https://towardsdatascience.com/getting-to-know-the-mel-spectrogram-31bca3e2d9d0
#https://arxiv.org/pdf/1809.04281.pdf
#https://towardsdatascience.com/transformers-141e32e69591
#http://cs229.stanford.edu/proj2017/final-reports/5244079.pdf
#https://musicinformationretrieval.com/chroma.html


In [ ]:
# file_path = '/Users/madhavapaliyam/Documents/CMSC/rando/raganet/examples/ex1.mp3'
file_path = '/Users/madhavapaliyam/Documents/CMSC/rando/raganet/data/AbhEri/03-vINAbhEri_vEnuvAdyAdi_vinOdinIm-AbhEri.mp3'
signal, sampling_rate = open_audio(file_path)
print(sampling_rate)

if len(signal.shape) > 1: 
    signal = np.mean(signal, axis = 1)
print(signal.shape)

In [ ]:
librosa.display.waveplot(signal)

In [ ]:
hop_length = 1024
n_chroma = 24

In [ ]:
S = librosa.cqt(signal, sr=sampling_rate, hop_length=hop_length)
# S_DB = librosa.power_to_db(S, ref=np.max)
print(S.shape)
librosa.display.specshow(S, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')

In [ ]:
mean_S = np.mean(S, axis = 0)
mean_S.shape

In [ ]:
chromagram = librosa.feature.chroma_stft(signal, sr=sampling_rate, n_chroma = n_chroma, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:], x_axis='time', hop_length=hop_length)
print(chromagram.shape)

In [ ]:
chromagram = librosa.feature.chroma_stft(signal, sr=sampling_rate, n_chroma = n_chroma, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:] * mean_S, x_axis='time', hop_length=hop_length)
print(chromagram.shape)

In [ ]:
chromagram = librosa.feature.chroma_cqt(signal, sr=sampling_rate, n_chroma = 12, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:], x_axis='time', hop_length=hop_length)
print(chromagram.shape)

In [ ]:
chromagram = librosa.feature.chroma_cqt(signal, sr=sampling_rate, n_chroma = 12, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:] * mean_S, x_axis='time', hop_length=hop_length)
print(chromagram.shape)

### Shifts

In [ ]:
# shifted_signal = librosa.effects.pitch_shift(signal, sampling_rate, n_steps=35)

In [ ]:
S = librosa.cqt(shifted_signal, sr=sampling_rate, hop_length=hop_length)
# S_DB = librosa.power_to_db(S, ref=np.max)
print(S.shape)
librosa.display.specshow(S, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')

In [ ]:
chromagram = librosa.feature.chroma_stft(shifted_signal, sr=sampling_rate, n_chroma = n_chroma, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:] * mean_S, x_axis='time', hop_length=hop_length)
print(chromagram.shape)

In [ ]:
chromagram = librosa.feature.chroma_cqt(shifted_signal, sr=sampling_rate, n_chroma = 12, hop_length=hop_length)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram[:,:] * mean_S, x_axis='time', hop_length=hop_length)
print(chromagram.shape)

In [ ]:
num_groups = chromagram.shape[1] // 600
bound = num_groups * 600
chromagram = chromagram[:,0:bound]
chromagram.shape

In [ ]:
np.array(np.split(chromagram, num_groups, axis = 1)).shape

In [ ]:
from nnAudio import Spectrogram

In [ ]:
signal_torch = torch.tensor(signal)
S = Spectrogram.CQT2010v2(hop_length = hop_length)(signal_torch)
S.shape

In [ ]:
librosa.display.specshow(S.cpu().detach().numpy().squeeze(), sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='mel')


In [ ]:
S_roll = torch.roll(S, -10, dims = 1)

In [ ]:
librosa.display.specshow(S_roll.cpu().detach().numpy().squeeze(), sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='mel')


### Filtering

In [ ]:
file_path = '/Users/madhavapaliyam/Documents/CMSC/rando/raganet/data/AbhEri/11-bhajarE_rE_mAnasa_shrI-AbhEri.mp3'
signal, sampling_rate = open_audio(file_path)
print(sampling_rate)

if len(signal.shape) > 1: 
    signal = np.mean(signal, axis = 1)
print(signal.shape)

S_full, phase = librosa.magphase(librosa.stft(signal))

In [ ]:
import IPython.display as ipd
# ipd.Audio(file_path)

In [ ]:
idx = slice(*librosa.time_to_frames([10, 15], sr=sampling_rate))
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(S_full[:, idx], ref=np.max),
                         y_axis='log', x_axis='time', sr=sampling_rate)
plt.colorbar()
plt.tight_layout()

In [ ]:
S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sampling_rate)))


In [ ]:
S_filter = np.minimum(S_full, S_filter)

In [ ]:
margin_i, margin_v = 5, 10
power = 2

mask_i = librosa.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)

mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)

In [ ]:
S_foreground = mask_v * S_full
S_background = mask_i * S_full

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(S_full[:, idx], ref=np.max),
                         y_axis='log', sr=sampling_rate)
plt.title('Full spectrum')
plt.colorbar()

plt.subplot(3, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S_background[:, idx], ref=np.max),
                         y_axis='log', sr=sampling_rate)
plt.title('Background')
plt.colorbar()
plt.subplot(3, 1, 3)
librosa.display.specshow(librosa.amplitude_to_db(S_foreground[:, idx], ref=np.max),
                         y_axis='log', x_axis='time', sr=sampling_rate)
plt.title('Foreground')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
import soundfile as sf
new_y = librosa.istft(S_foreground*phase)
sf.write("./new-audio.wav", new_y, sampling_rate)

In [ ]:
# ipd.Audio("./new-audio.wav")